In [1]:
import torch
from torch import nn

import shutil

from compressai.zoo import load_state_dict, models

import loralib as lora

In [2]:
def load_checkpoint(arch: str, checkpoint_path: str, strict=True) -> nn.Module:
    state_dict = load_state_dict(
        torch.load(checkpoint_path, map_location=torch.device("cpu"))["state_dict"]
    )
    return models[arch].from_state_dict(state_dict, strict)


In [3]:
pretrain_ckpt = "./ckpt/stf_0018.pth.tar"
lora_ckpt = "./ckpt/lora.pth.tar"

In [4]:
lora_model = load_checkpoint("lora_stf", pretrain_ckpt, False)
lora_state = torch.load(lora_ckpt)
lora_model.load_lora_state(lora_state)

/Users/Nero/miniconda3/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
for n, p in lora_model.named_parameters():
    print(n)

patch_embed.proj.weight
patch_embed.proj.bias
patch_embed.norm.weight
patch_embed.norm.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.attn.relative_position_bias_table
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.qkv.lora_A
layers.0.blocks.0.attn.qkv.lora_B
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.attn.proj.lora_A
layers.0.blocks.0.attn.proj.lora_B
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.blocks.0.mlp.fc1.lora_A
layers.0.blocks.0.mlp.fc1.lora_B
layers.0.blocks.0.mlp.fc2.weight
layers.0.blocks.0.mlp.fc2.bias
layers.0.blocks.0.mlp.fc2.lora_A
layers.0.blocks.0.mlp.fc2.lora_B
layers.0.blocks.1.norm1.weight
layers.0.blocks.1.norm1.bias
layers.0.blocks.1.attn.relative_position_bias_table
layers.0.blocks.1.attn.qkv.weight
layers.0.blocks.1.attn.qkv.bias
layers.0.blocks.1.attn.qkv

In [6]:
lora.mark_only_lora_as_trainable(lora_model)
lora_params_cnt = sum(p.numel() for p in lora_model.parameters() if p.requires_grad == True )
model_params_cnt = sum(p.numel() for p in lora_model.parameters() if p.requires_grad == False )
print(lora_params_cnt)
print(model_params_cnt)
print(lora_params_cnt / model_params_cnt * 100)

6098136
99855639
